##### yes24 장르/나이/성별 베스트셀러 크롤링

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
}

def get_yes24_bestsellers(category_number=None, sex=None, age=None, max_items=10):
    
    # 카테고리가 없으면 종합 베스트셀러 가져오기 - 전체: 001
    category_number = category_number or "001"
    url = f'https://www.yes24.com/Product/Category/DayBestSeller?categoryNumber={category_number}'
    
    # 성별 및 나이 조건 추가
    if sex: url += f"&sex={sex}"
    if age: url += f"&age={age}"

    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    book_items = soup.select("div.itemUnit")[:max_items]
    book_list = []
    
    for item in book_items:
        try:
            rank = item.select_one(".ico.rank").text
            title_tag = item.select_one(".gd_name")
            author = item.select_one("a[href*='author=']").text
            publisher = item.select_one("a[href*='company=']").text
            date = item.select_one(".authPub.info_date").text

            if not all([rank, title_tag, author, publisher, date]):
                continue

            title = title_tag.text.strip()
            link = "https://www.yes24.com" + title_tag["href"]
            
            book_list.append({
                "순위": rank,
                "제목": title,
                "저자": author,
                "출판사": publisher,
                "출간일": date,
                "링크": link
            })
        except Exception:
            continue

    return pd.DataFrame(book_list)

In [19]:
# 국내도서 - 건강 취미, 여자, 20대 설정
health_category = "001001011"
df_health = get_yes24_bestsellers(health_category, 'F', 20)
print("📘 이북 베스트셀러")
df_health

📘 이북 베스트셀러


,순위,제목,저자,출판사,출간일,링크
0,1,쿵야 레스토랑즈 초스페셜 스티커북,쿵야 레스토랑즈,다산북스,2025년 04월,https://www.yes24.com/product/goods/144892635
1,2,블루밍니트의 코바늘 플라워&키링,블루밍니트,동양북스(동양books),2025년 04월,https://www.yes24.com/product/goods/144875040
2,3,일본 현지 아이스크림 대백과,아이스맨 후쿠토메,클,2023년 11월,https://www.yes24.com/product/goods/123687533
3,4,크툴루의 부름 : 수호자 룰북,샌디 피터슨,초여명,2016년 11월,https://www.yes24.com/product/goods/33213078
4,5,굿 스트레스 GOOD STRESS,이한성,미문사,2025년 04월,https://www.yes24.com/product/goods/145321947
5,6,야구×수학,류선규,페이스메이커,2025년 04월,https://www.yes24.com/product/goods/145220263
6,7,혼자서 국내 여행,이학균,디스커버리미디어,2025년 04월,https://www.yes24.com/product/goods/143604048
7,8,디스 이즈 이탈리아,전혜진,테라(TERRA),2025년 03월,https://www.yes24.com/product/goods/143253894
8,9,세상에서 가장 알기 쉬운 근육연결도감 : 셀프케어편,키마타 료,중앙북스(books),2025년 03월,https://www.yes24.com/product/goods/143088088
9,10,Report 03 : Sweet Delusion,미고,은행나무,2025년 03월,https://www.yes24.com/product/goods/142813809
